In [1]:
import weaviate
import json
client = weaviate.Client('http://localhost:8080')
schema = client.schema.get()
print(schema)

{'classes': []}


In [2]:
schema = {
    "classes": [
        {
            "class": "Dog",
            "description": "Images of different dogs",
            "moduleConfig": {
                "img2vec-neural": {
                    "imageFields": [
                        "image"
                    ]
                }
            },
            "vectorIndexType": "hnsw", 
            "vectorizer": "img2vec-neural", # the img2vec-neural Weaviate module
            "properties": [
                {
                    "name": "breed",
                    "dataType": ["string"],
                    "description": "name of dog breed",
                },
                {
                    "name": "image",
                    "dataType": ["blob"],
                    "description": "image",
                },
                {
                    "name": "filepath",
                    "dataType":["string"],
                    "description": "filepath of the images",
                }
            ]
        }
    ]
}

# adding the schema 
client.schema.create(schema)

In [3]:
schema = client.schema.get()
display(schema)

{'classes': [{'class': 'Dog',
   'description': 'Images of different dogs',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'img2vec-neural': {'imageFields': ['image']}},
   'properties': [{'dataType': ['string'],
     'description': 'name of dog breed',
     'moduleConfig': {'img2vec-neural': {}},
     'name': 'breed',
     'tokenization': 'word'},
    {'dataType': ['blob'],
     'description': 'image',
     'moduleConfig': {'img2vec-neural': {}},
     'name': 'image'},
    {'dataType': ['string'],
     'description': 'filepath of the images',
     'moduleConfig': {'img2vec-neural': {}},
     'name': 'filepath',
     'tokenization': 'word'}],
   'replicationConfig': {'factor': 1},
   'shardingConfig': {'virtualPerPhysical': 128,
    'desiredCount': 1,
    'actualCount': 1,
    'desiredVirtualCount': 128,
    'actualVirtualCount': 128,
    'key': '_i

In [4]:
from pathlib import Path
import base64


img_path = Path("./flask-app/static/img/")
encoded_path = Path("base64_images")
encoded_path.mkdir(exist_ok=True)

for file_path in img_path.iterdir():
    with file_path.open("rb") as f:
        encoded = base64.b64encode(f.read())
    with (encoded_path / file_path.stem).open("wb+") as f:
        f.write(encoded)


print("The images have been converted to base64.")

The images have been converted to base64.


In [5]:
client.batch.configure(
    batch_size=100, 
    dynamic=True,
    timeout_retries=3,
    callback=None,
)
    
with client.batch as batch:
    batch.delete_objects(
        class_name="Dog",
        # same where operator as in the GraphQL API
        where={
            "operator": "NotEqual",
            "path": ["breed"],
            "valueString": "x"
        },
        output="verbose",
    )

encoded_path = Path("base64_images")
with client.batch as batch:
    # Iterate over all files in the base64_images folder
    for encoded_file_path in encoded_path.iterdir():
        with encoded_file_path.open() as f:
            file = f.read()

        base64_encoding = file.replace("\n", "").replace(" ", "") 

        # The properties from our schema
        data_properties = {
            "breed": encoded_file_path.stem,
            "image": base64_encoding,
            "filepath": encoded_file_path.stem + ".jpg",
        }

        batch.add_data_object(data_properties, "Dog")

print("The objects have been uploaded to Weaviate.")

The objects have been uploaded to Weaviate.


In [6]:
client.query.get(
    class_name="Dog",
    properties=["filepath", "breed", "image"]
).do()


{'data': {'Get': {'Dog': [{'breed': 'Labrador-Retriever',
     'filepath': 'Labrador-Retriever.jpg',
     'image': '...'},
    {'breed': 'Rottweiler',
     'filepath': 'Rottweiler.jpg',
     'image': '...'},
    {'breed': 'Australian-Shepherd',
     'filepath': 'Australian-Shepherd.jpg',
     'image': '...'},
    {'breed': 'Golden-Retriever',
     'filepath': 'Golden-Retriever.jpg',
     'image': '...'},
    {'breed': 'Siberian-Husky',
     'filepath': 'Siberian-Husky.jpg',
     'image': '...'},
    {'breed': 'French-Bulldog',
     'filepath': 'French-Bulldog.jpg',
     'image': '...'},
    {'breed': 'Bernese-Mountain-Dog',
     'filepath': 'Bernese-Mountain-Dog.jpg',
     'image': '...'},
    {'breed': 'Goldendoodle',
     'filepath': 'Goldendoodle.jpg',
     'image': '...'},
    {'breed': 'German-Shepherd',
     'filepath': 'German-Shepherd.jpg',
     'image': '...'},
    {'breed': 'Corgi',
     'filepath': 'Corgi.jpg',
     'image': '...'}]}}}

In [7]:
client.data_object.get(with_vector=True)

{'deprecations': None,
 'objects': [{'class': 'Dog',
   'creationTimeUnix': 1681471259044,
   'id': '01e3c2a2-7d4d-41e6-a19b-5ceb57dc7190',
   'lastUpdateTimeUnix': 1681471259044,
   'properties': {'breed': 'Labrador-Retriever',
    'filepath': 'Labrador-Retriever.jpg',
    'image': '...'},
   'vector': [0.7738535,
    1.3056467,
    2.0999842,
    0.06647085,
    0.47060913,
    0.28797865,
    0.5761432,
    0.58936983,
    0.16387363,
    0.2596277,
    0.3773572,
    0.12693894,
    0.74769956,
    0.5338556,
    0.13069364,
    0.25196,
    0.26281387,
    0.12510909,
    0.083481245,
    0.3460368,
    0.05754356,
    0.6560033,
    0.4674056,
    0.13423483,
    0.8079887,
    0.3388179,
    0.17556535,
    0.9257013,
    0.21812253,
    0.6236684,
    0.30813432,
    0.39284253,
    0.7028544,
    0.134232,
    0.18141411,
    0.3388376,
    0.4110018,
    1.6805971,
    0.28099036,
    1.0254365,
    0.23311776,
    0.64437115,
    0.25394484,
    0.1230521,
    0.22689442,
  